## How to locally run parallel code with mpi4py in an IPython notebook:

The prerequisite for this is a working installation of some MPI distribution.

Using Ubuntu or some derivative, I recommend using OpenMPI which can be istalled from the repository by means of the following packages: *libopenmpi-dev, openmpi-bin, openmpi-doc*.

Now, you can already run MPI enabled code from you shell by calling

*$mpirun -n [numbmer_of_threads] python [script_to_run.py]*

To use MPI with iPython, one has to install ipyparallel:

via pip: *$pip install ipyparallel*

via conda: *$conda install ipyparallel*

and then enable the Clusters tab in ipython via

*$ ipcluster nbextension enable*

To make MPI acessable via mpi4py in an ipython notebook, one has to do the following:
open one shell and start the ipcontroller:

*$ipcontroller*

open another shell and start a number of engines:

*$mpirun -n [number of engines] ipengine --mpi=mpi4py*

and then connect to the engines via the following fragment of code:

In [11]:
from ipyparallel import Client
import os
c = Client()
view = c[:]
print(c.ids)

[0, 1, 2, 3]


In [5]:
%%px

def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return root
path = find('02_LocalParallelization.ipynb', '/home/')
print(path)
os.chdir(path)

[stdout:0] /home/jakob/Project_PyMofa/tutorial
[stdout:1] /home/jakob/Project_PyMofa/tutorial
[stdout:2] /home/jakob/Project_PyMofa/tutorial
[stdout:3] /home/jakob/Project_PyMofa/tutorial


Now, to make the code run on all of our engines (and not just on one), the following cells have to start with the [__parallel magic__](https://ipython.org/ipython-doc/3/parallel/magics.html) command *%%px*

In [6]:
%%px
from mpi4py import MPI
com = MPI.COMM_WORLD
print(com.Get_rank())

[stdout:0] 3
[stdout:1] 1
[stdout:2] 0
[stdout:3] 2


Now, that we have MPI running, and mpi4py recognizing the nodes and their ranks, we can continue with the predator prey exercise, that we know from the first tutorial.

First, define the model:

In [7]:
%%px
import numpy as np
import matplotlib.pyplot as plt

def predprey_model(prey_birth_rate=0.1, prey_mortality=0.1, 
                   predator_efficiency=0.1, predator_death_rate=0.01,
                   initial_prey=1., initial_predators=1.,
                   time_length=1000):
    """Discrete predetor prey model."""
    A = -1 * np.ones(time_length)
    B = -1 * np.ones(time_length)
    A[0] = initial_prey
    B[0] = initial_predators
    for t in range(1, time_length):
        A[t] = A[t-1] + prey_birth_rate * A[t-1] - prey_mortality * B[t-1]*A[t-1]
        B[t] = B[t-1] + predator_efficiency * B[t-1]*A[t-1] - predator_death_rate * B[t-1] +\
            0.02 * (0.5 - np.random.rand())
    return A, B

#preys, predators = predprey_model()
#plt.plot(preys, label="preys") 
#plt.plot(predators, label="predators")
#plt.legend()
#plt.show()

Then import the experiment_handling class from pymofa and define a run function:

In [8]:
%%px
# imports
from pymofa.experiment_handling import experiment_handling as eh
import itertools as it
import pandas as pd
# import cPickle


#Definingh the experiment execution function
#      it gets paramater you want to investigate, plus `filename` as the last parameter
def RUN_FUNC(prey_birth_rate=0.1, prey_mortality=0.1, 
                   predator_efficiency=0.1, predator_death_rate=0.01,
                   initial_prey=1., initial_predators=1.,
                   time_length=1000, filename='./'):
    """Insightful docstring."""
    print(prey_birth_rate, prey_mortality, 
                   predator_efficiency, predator_death_rate,
                   initial_prey, initial_predators,
                   time_length)
    # one could also do more complicated stuff here, e.g. drawing something from a random distribution
    
    # running the model
    # TO DO: there seems to be a problem passing arguments to function
    #preys, predators = predprey_model(prey_birth_rate, prey_mortality, 
    #                                  predator_efficiency, predator_death_rate,
    #                                  initial_prey, initial_predators,
    #                                  time_length)
    preys, predators = predprey_model(
    )
    print(preys)
    # preparing the data
    res = pd.DataFrame({"preys": np.array(preys),
                        "predators": np.array(predators)})
    
    # Save Result
    res.to_pickle(filename)
    
    # determine exit status (if something went wrong)
    # if exit status > 0 == run passen
    # if exit status < 0 == Run Failed
    exit_status = 1
    
    # RUN_FUNC needs to return exit_status
    return exit_status

Specify the necessary parameters, generate their combinations and feed them to an experiment handle:

In [9]:
%%px
# Path where to Store the simulated Data
SAVE_PATH_RAW = "./dummy/pymofatutorial"

# Parameter combinations to investiage

prey_birth_rate = [0.1]
predator_death_rate = [0.1]
initial_pop = [1.]
            
PARAM_COMBS = list(it.product(prey_birth_rate, predator_death_rate, initial_pop))

# Sample Size
SAMPLE_SIZE = 5

# INDEX 
INDEX = {0: 'prey_birth_rate', 1: 'predator_death_rate', 2: 'initial_prey'}

# initiate handle instance with experiment variables
handle = eh(SAMPLE_SIZE, PARAM_COMBS, INDEX, SAVE_PATH_RAW)

[stdout:0] initializing pymofa experiment handle
[stdout:1] initializing pymofa experiment handle
[stdout:2] 
initializing pymofa experiment handle
detected 4 nodes in MPI environment
[stdout:3] initializing pymofa experiment handle


And finally run the model - now in parallel:

In [10]:
%%px
# Compute experiemnts raw data
handle.compute(RUN_FUNC)

[stdout:0] 
0.1 0.1 1.0 0.01 1.0 1.0 1000
[  1.00000000e+00   1.00000000e+00   9.91674642e-01   9.73540145e-01
   9.45090242e-01   9.06516153e-01   8.58956476e-01   8.03209299e-01
   7.40500972e-01   6.73276286e-01   6.03923016e-01   5.34879696e-01
   4.67860021e-01   4.04240508e-01   3.45855404e-01   2.93634338e-01
   2.47716789e-01   2.07957307e-01   1.73900593e-01   1.44821540e-01
   1.20421287e-01   1.00028709e-01   8.30345959e-02   6.89912691e-02
   5.73557909e-02   4.77819148e-02   3.98866282e-02   3.33716112e-02
   2.79524338e-02   2.34557856e-02   1.97212527e-02   1.66081283e-02
   1.40159792e-02   1.18713541e-02   1.00916056e-02   8.60402157e-03
   7.35643709e-03   6.30050854e-03   5.41390236e-03   4.65902474e-03
   4.02238618e-03   3.48041695e-03   3.01922654e-03   2.62515644e-03
   2.29038567e-03   2.00356994e-03   1.75774133e-03   1.54593341e-03
   1.36245664e-03   1.20336766e-03   1.06546355e-03   9.45430119e-04
   8.40980551e-04   7.50490760e-04   6.71864232e-04   6.02504

And if everyting whent well, the calculations should have been splitted between all the engines that you've started in the beginning.

To run you experiments in scripts outside of an IPython notebook, simply run you experiment script (defining a run function, an experiment handle and calling the compute routine of that handle) with mpirun in a terminal 